In [77]:
import os
import pandas as pd

from datetime import datetime
from utils.config import DRLConfig
from utils.drl_train import training_pipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
df_ret = pd.read_parquet("../data/returns.parquet")
df_prices = pd.read_parquet("../data/prices.parquet")
df_vol = pd.read_parquet("../data/vola.parquet")

To view the logs:
1. Open a terminal or command prompt.
2. Navigate to the directory *containing* the `logs` directory (i.e., the root of this repository).
3. Run the command: `tensorboard --logdir logs/`
4. Open the URL provided by TensorBoard (usually http://localhost:6006/) in your web browser.

You should see experiments named like `PPO_WindowX_AgentY_SeedZ`.

In [79]:
# Create timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create configuration
config = DRLConfig(
    # Window configuration
    n_windows=1,  # 10 in paper
    agents_per_window=5,  # 5 in paper
    base_start_year=2006,
    # Environment parameters
    env_window_size=60,
    transaction_cost=0.0,
    initial_balance=100_000,
    reward_scaling=1.0,
    eta_dsr=1 / 252,
    # Training parameters
    n_envs=10,
    total_timesteps_per_round=7_500_000,  # 7_500_000 in paper
    n_steps_per_env=252 * 3,
    batch_size=1260,
    n_epochs=16,
    gamma=0.9,
    gae_lambda=0.9,
    clip_range=0.25,
    log_std_init=-1.0,
    # Learning rate parameters
    initial_lr=3e-4,
    final_lr=1e-5,
    # Paths
    model_save_dir=f"../models/{timestamp}",
    tensorboard_log_dir=f"../logs/{timestamp}",
)

In [80]:
# Run training pipeline
results, backtest_portfolio = training_pipeline(
    drl_config=config, df_prices=df_prices, df_ret=df_ret, df_vol=df_vol
)

--- Starting Window 1/1 (Train Year Start: 2006) ---
  Train Period: 2006-01-01 to 2010-12-31
  Val Period  : 2011-01-01 to 2011-12-31
  Test Period : 2012-01-01 to 2012-12-31
  Training Agent 1/5 with seed 0...


Output()

    Starting training for 7500000 timesteps...


KeyboardInterrupt: 

In [ ]:
backtest_portfolio[0].get_history()

,portfolio_value,cash,w_c,w_XLF,w_XLK,w_XLV,w_XLY,w_XLP,w_XLE,w_XLI,...,s_XLK,s_XLV,s_XLY,s_XLP,s_XLE,s_XLI,s_XLU,s_XLB,s_XLRE,s_XLC
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,100000.000000,18013.456625,0.180135,0.0923,0.0959,0.0892,0.0892,0.0892,0.0894,0.0892,...,445,316,266,391,201,332,396,366,0,0
2012-01-04,100096.772869,17969.025246,0.179517,0.0928,0.0961,0.0889,0.0892,0.0892,0.0891,0.0890,...,445,317,264,392,200,330,397,368,0,0
2012-01-05,100352.195221,18001.580901,0.179384,0.0925,0.0965,0.0890,0.0890,0.0891,0.0888,0.0891,...,447,317,262,393,201,331,397,369,0,0
2012-01-06,100138.355886,17983.685228,0.179588,0.0926,0.0964,0.0890,0.0890,0.0892,0.0889,0.0891,...,445,316,261,395,202,331,399,367,0,0
2012-01-09,100330.063349,17814.694003,0.177561,0.0937,0.0997,0.0882,0.0880,0.0880,0.0879,0.0880,...,462,313,259,390,199,325,394,382,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-21,111301.597693,19491.746186,0.175125,0.0956,0.1034,0.0868,0.0865,0.0868,0.0867,0.0867,...,468,293,235,379,210,319,411,408,0,0
2012-12-24,111107.651207,19427.252551,0.174851,0.0959,0.1044,0.0863,0.0864,0.0864,0.0862,0.0863,...,473,292,234,378,210,318,411,409,0,0
2012-12-26,110743.850407,19475.607354,0.175862,0.0951,0.1021,0.0872,0.0872,0.0871,0.0869,0.0871,...,464,295,238,383,212,321,417,395,0,0


In [76]:
results_filename = f"backtest_results_summary_{timestamp}.csv"
results_save_path = os.path.join(config.model_save_dir, results_filename)

results_df = pd.DataFrame(results)
results_df.to_csv(results_save_path, index=False)
print(f"\nBacktest results summary saved to: {results_save_path}")
print("\nFinal Results DataFrame:")
results_df.head()


Backtest results summary saved to: ../models/20250603_003619/backtest_results_summary_20250603_003619.csv

Final Results DataFrame:


,window,best_agent_path,n_eval_episodes,mean_reward,std_reward,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk (95%),Portfolio turnover (in %),final_portfolio_value_first_episode
0,1,agent_seed0_valrew-10,1,-19.423373,0.0,0.097805,0.09618,0.099488,0.987823,1.350245,0.909514,-0.072435,1.181739,1.525845,0.043187,0.782472,1.158062,-0.009649,1.169297,109618
